# 1. Load Libary

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
import plotly.graph_objects as go
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers, Input
from tensorflow.keras import losses, optimizers
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt

# 2. Read csv file and Preprocessing

In [ ]:
train_df = pd.read_csv("../input/mobile-price-classification/train.csv")

In [ ]:
print(train_df.isnull().sum(axis=0))
print(train_df.info())

In [ ]:
train = train_df.iloc[:, :-1]
label = train_df.iloc[:, -1]

In [ ]:
train.head()

# Below graph, you can check correration between variables

In [ ]:
corr = train.corr()

fig = go.Figure()
fig.add_trace(go.Heatmap(z=corr.values, x=corr.index.values, y=corr.columns.values))
fig.show()

# Apply MinMaxScaler and Randomforest algorithms

In [ ]:
mc = MinMaxScaler()
mc.fit(train)
train = mc.transform(train)

In [ ]:
n_esti = list(range(5,16))
max_dep = list(range(5,16))

In [ ]:
rf = RandomForestClassifier()
rf_param_grid = {'n_estimators' : n_esti, 'max_depth' : max_dep}
rf_grid = GridSearchCV(estimator=rf,
                      param_grid=rf_param_grid,
                      scoring='accuracy',
                      cv=4,
                      return_train_score=True)

In [ ]:
rf_grid.fit(train,label)

# Check Result (Train : 99%, Test : 86%)
> best parameter is max_depth:15, n_estimator:15

In [ ]:
rf_grid_df = pd.DataFrame(rf_grid.cv_results_)
rf_grid_df.loc[rf_grid_df['rank_test_score'] == 1, ]

# Using keras simple DNN

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, label, test_size=0.33, random_state=10, stratify=label)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

input_shape = X_train.shape[1]

In [ ]:
input_tensor = Input(shape=(input_shape, ))
x = layers.Dense(128, activation='relu')(input_tensor)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dense(8, activation='relu')(x)
x = layers.Dense(4, activation='relu')(x)
output_tensor = layers.Dense(4, activation='softmax')(x)
model = Model(input_tensor, output_tensor)

In [ ]:
model.compile(optimizer=optimizers.Adam(), loss=losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
callback_list = [
    EarlyStopping(monitor='accuracy',patience=2),
    ReduceLROnPlateau(monitor='loss', factor=0.1, patience=1)
]

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    callbacks=callback_list, batch_size =8, epochs=100)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc)+1)

plt.figure()

plt.plot(epochs, acc, 'bo', label='Train Acc')
plt.plot(epochs, val_acc, 'b', label='Val Acc')
plt.title('Training and validation Acc')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Train Loss')
plt.plot(epochs, val_loss, 'b', label='Val Loss')
plt.title('Training and validation Loss')
plt.legend()

plt.show()